In [ ]:
#importing libraries
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import Concatenate
import numpy as np
from numpy import expand_dims


from matplotlib import pyplot as plt

In [ ]:
(trainX, trainy), (testX, testy) = load_data()

In [ ]:
# function to define discriminator
def define_discriminator(in_shape=(28,28,1), n_classes=10):
	# Input for class labels
	in_label = Input(shape=(1,))
	# Embedding for label input
	li = Embedding(n_classes, 50)(in_label)
	# Scale up to image dimensions with linear activation
	n_nodes = in_shape[0] * in_shape[1]
	li = Dense(n_nodes)(li)
	# Reshape to additional channel
	li = Reshape((in_shape[0], in_shape[1], 1))(li)
	# Input for image
	in_image = Input(shape=in_shape)
	# Concatenate image and label embeddings
	merge = Concatenate()([in_image, li])
	# Convolutional layers with leaky relu
	fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(merge)
	fe = LeakyReLU(alpha=0.2)(fe)
	fe = Conv2D(128, (3,3), strides=(2,2), padding='same')(fe)
	fe = LeakyReLU(alpha=0.2)(fe)
	# Flatten feature maps
	fe = Flatten()(fe)
	# Dropout layer for regularization
	fe = Dropout(0.4)(fe)
	# Output layer with sigmoid activation
	out_layer = Dense(1, activation='sigmoid')(fe)
	# Model
	model = Model([in_image, in_label], out_layer)
	# Compile model with binary cross-entropy loss and Adam optimizer
	opt = Adam(learning_rate=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	return model


In [ ]:
def define_generator(latent_dim, n_classes=10):
	# Input for class labels
	in_label = Input(shape=(1,))
	# Embedding for label input
	li = Embedding(n_classes, 50)(in_label)
	# Scale up to image dimensions with linear activation
	n_nodes = 7 * 7
	li = Dense(n_nodes)(li)
	# Reshape to additional channel
	li = Reshape((7, 7, 1))(li)
	# Input for noise
	in_lat = Input(shape=(latent_dim,))
	# Number of nodes
	n_nodes = 128 * 7 * 7
	# Linear activation
	gen = Dense(n_nodes)(in_lat)
	# Apply leaky relu
	gen = LeakyReLU(alpha=0.2)(gen)
	# Reshape to additional channel
	gen = Reshape((7, 7, 128))(gen)
	# Merge image gen and label input
	merge = Concatenate()([gen, li])
	# Upsample
	gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(merge)
	# Apply leaky relu
	gen = LeakyReLU(alpha=0.2)(gen)
	# Upsample
	gen = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(gen)
	# Apply leaky relu
	gen = LeakyReLU(alpha=0.2)(gen)
	# Output
	out_layer = Conv2D(1, (7,7), activation='tanh', padding='same')(gen)
	# Model
	model = Model([in_lat, in_label], out_layer)
	return model


In [ ]:
def define_gan(g_model, d_model):
	# Make discriminator not trainable
	d_model.trainable = False
	# Input for generator (noise and labels)
	gen_noise, gen_label = g_model.input
	# Output from generator
	gen_output = g_model.output
	# Output from discriminator given generated images and labels
	gan_output = d_model([gen_output, gen_label])
	# Define GAN model with generator inputs and discriminator output
	model = Model([gen_noise, gen_label], gan_output)
	# Compile GAN with Adam optimizer and binary cross-entropy loss
	opt = Adam(learning_rate=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model


In [ ]:
def load_real_samples():
    # Load MNIST dataset
    (trainX, trainy), (testX, testy) = load_data()
    # Expand dimensions of trainX to include channel dimension
    X = expand_dims(trainX, axis=-1)
    # Convert to float32
    X = X.astype('float32')
    # Normalize to range [-1, 1]
    X = (X - 127.5) / 127.5
    # Return normalized images and corresponding labels
    return [X, trainy]


In [ ]:
def generate_real_samples(dataset, n_samples):
    # Unpack dataset into images and labels
    images, labels = dataset
    # Randomly select indices
    ix = randint(0, images.shape[0], n_samples)
    # Select images and corresponding labels based on indices
    X, labels = images[ix], labels[ix]
    # Generate 'real' class labels (1 for real)
    y = ones((n_samples, 1))
    # Return selected images and labels with 'real' labels
    return [X, labels], y


In [ ]:
def generate_latent_points(latent_dim, n_samples, n_classes=10):
    # Generate random input for latent space
    x_input = randn(latent_dim * n_samples)
    # Reshape into a batch of inputs for the generator
    z_input = x_input.reshape(n_samples, latent_dim)
    # Generate random labels
    labels = randint(0, n_classes, n_samples)
    # Return generated latent points and labels
    return [z_input, labels]


In [ ]:
def generate_fake_samples(generator, latent_dim, n_samples):
    # Generate latent points and random labels
    z_input, labels_input = generate_latent_points(latent_dim, n_samples)
    # Generate fake images using the generator
    images = generator.predict([z_input, labels_input])
    # Generate 'fake' class labels (0 for fake)
    y = zeros((n_samples, 1))
    # Return generated fake images with corresponding labels
    return [np.array(images), labels_input], y


In [ ]:
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=128):
    # Calculate number of batches per epoch
    bat_per_epo = int(dataset[0].shape[0] / n_batch)
    # Half batch size for each real and fake samples
    half_batch = int(n_batch / 2)

    # Iterate over epochs
    for i in range(n_epochs):
        # Iterate over batches
        for j in range(bat_per_epo):
            # Train discriminator on real samples
            [X_real, labels_real], y_real = generate_real_samples(dataset, half_batch)
            d_loss_real, _ = d_model.train_on_batch([X_real, labels_real], y_real)

            # Train discriminator on fake samples
            [X_fake, labels], y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            d_loss_fake, _ = d_model.train_on_batch([X_fake, labels], y_fake)

            # Train generator via GAN model
            [z_input, labels_input] = generate_latent_points(latent_dim, n_batch)
            y_gan = ones((n_batch, 1))
            g_loss = gan_model.train_on_batch([z_input, labels_input], y_gan)

            # Print progress
            print('Epoch>%d, Batch%d/%d, d1=%.3f, d2=%.3f g=%.3f' %
                  (i+1, j+1, bat_per_epo, d_loss_real, d_loss_fake, g_loss))

    # Save the generator model after training
    g_model.save('mnist_conditional_generator.h5')


In [ ]:

latent_dim = 100
d_model = define_discriminator()
g_model = define_generator(latent_dim)
gan_model = define_gan(g_model, d_model)
dataset = load_real_samples()
train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100,n_batch=128)

In [ ]:

from numpy import asarray
from numpy.random import randn
from numpy.random import randint
from keras.models import load_model


In [ ]:

model = load_model('mnist_conditional_generator.h5')

In [ ]:
#plotting the results
latent_points, labels = generate_latent_points(100, 100)
labels = asarray([x for _ in range(10) for x in range(10)])
X  = model.predict([latent_points, labels])
X = (X + 1) / 2.0
X = (X*255).astype(np.uint8)
def show_plot(examples, n):
	for i in range(n * n):
		plt.subplot(n, n, 1 + i)
		plt.axis('off')
		plt.imshow(examples[i, :, :, :],cmap='gray_r')
	plt.show()

show_plot(X, 10)